In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from random import randrange
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tqdm import tqdm

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255
test_images = test_images / 255

In [3]:
feature_count = train_images.shape[1] * train_images.shape[2]
class_count = len(set(train_labels))

In [4]:
def result_vector(labels, object_id: int):
    class_id = labels[object_id]
    answer = np.zeros(class_count, dtype=np.float64)
    answer[class_id] = 1
    return answer

def feature_vector(images, object_id: int):
    flatten = images[object_id].reshape(feature_count)
    return np.append(flatten, [1])  # add bias feature

In [5]:
def get_random_samples_x_y():
    # currently batch size = 1

    object_id = randrange(len(train_images))
    return [feature_vector(train_images, object_id)], [result_vector(train_labels, object_id)]

In [11]:
maximum_steps = 10000
step_factor = 0.1

x = tf.placeholder(tf.float64, [None, feature_count + 1])
y = tf.placeholder(tf.float64, [None, class_count])

W = tf.Variable(tf.ones([feature_count + 1, class_count], tf.float64))

predictions = tf.nn.softmax(tf.matmul(x, W))
cost = tf.nn.sigmoid_cross_entropy_with_logits(logits=predictions, labels=y)

g = tf.gradients(cost, W)[0]
update_W = W.assign_sub(step_factor * g)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for steps in tqdm(range(1, maximum_steps + 1)):
        features, answers = get_random_samples_x_y()

        sess.run(update_W, feed_dict={x: features, y: answers})

    W_trained = sess.run(W)

100%|██████████| 10000/10000 [00:14<00:00, 700.17it/s]


In [12]:
def get_random_test_x_label():
    object_id = randrange(len(test_images))
    return [feature_vector(test_images, object_id)], test_labels[object_id]

def get_label(predicted):
    return np.argmax(predicted)


correct = 0
predictions_count = 10000

with tf.Session() as sess:
    sess.run(W.assign(W_trained))

    for i in tqdm(range(predictions_count)):
        features, answer_label = get_random_test_x_label()
        predicted = sess.run(predictions, feed_dict={x: features})

        predicted_label = get_label(predicted)

        if answer_label == predicted_label:
            correct += 1

print("correct ratio:", correct / predictions_count)

100%|██████████| 10000/10000 [00:11<00:00, 871.68it/s]


correct ratio: 0.896
